In [1]:
from PIL import Image
import torch
import torch.nn as nn
from torchvision import models
from torchvision import datasets, transforms, models
import torch.optim as optim
from torch.utils.data import DataLoader
INPUT_SIZE=256
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
import cv2
import subprocess

In [2]:
test_transforms = transforms.Compose([
    transforms.Resize((INPUT_SIZE, INPUT_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_dataset = datasets.ImageFolder("../../data/test", transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [3]:
# Load the model architecture
model = models.mobilenet_v2(pretrained=False)
num_classes = len(test_dataset.classes)
model.classifier[1] = nn.Linear(model.last_channel, num_classes)

# Load the state_dict onto the CPU
state_dict = torch.load('../../model/model-01/mobilenet_classification.pth', map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

# Set the model to evaluation mode
model.eval()
model=model.to(device)
print("Model successfully loaded on CPU!")

Model successfully loaded on CPU!


/Users/vobahoangnhat/anaconda3/envs/min_ds-env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/vobahoangnhat/anaconda3/envs/min_ds-env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler (tùy chọn)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [5]:
# model.eval()
# test_loss = 0.0
# correct = 0
# total = 0
# with torch.no_grad():
#     for inputs, labels in test_loader:
#         inputs, labels = inputs.to(device), labels.to(device)

#         outputs = model(inputs)
#         loss = criterion(outputs, labels)

#         _, predicted = outputs.max(1)
#         total += labels.size(0)
#         correct += predicted.eq(labels).sum().item()
#         test_loss += loss.item()

#     val_accuracy = 100. * correct / total
#     print(f'Test Loss: {test_loss/len(test_dataset):.4f}, Accuracy: {val_accuracy:.2f}%')

In [6]:
def predict_image(image_path, model, transform, classes):
    model.eval()
    image = cv2.imread(image_path)
    
    # Resize
    resized_img = cv2.resize(image, (256, int(image.shape[0] * 256 / image.shape[1])))
    # Cut
    cropped_img = resized_img[:256, :256]
    cv2.imshow("../../data/query/1_.jpg",cropped_img)
    # Convert to numpy array to PIL image
    cropped_img = Image.fromarray(cropped_img)

    cropped_img = transform(cropped_img).unsqueeze(0)

    with torch.no_grad():
        outputs = model(cropped_img)
        _, predicted = outputs.max(1)
        return classes[predicted.item()]

# Ví dụ dự đoán
classes = test_dataset.classes
img=['1.jpg','2.jpg','3.jpg','4.jpg','5.jpg','6.jpg']
query_path='../../data/query'
for imgage in img:
    image_path =query_path+"/"+imgage
    prediction = predict_image(image_path, model, test_transforms, classes)
    print(f'Predicted class: {prediction}')
    if prediction=='0':
        # Change query_path
        query_path_loc='../Model-Selection/data/query'
        # Thư mục làm việc
        working_directory = "../../../Hierarchical-Localization/"

        # Lệnh và tham số
        command = [
            "python3",
            "outdoor_localize.py",
            "--folder_query", query_path_loc,
            "--query_name", imgage
        ]

        # # Thực thi và thu thập kết quả
        # result = subprocess.run(command, cwd=working_directory, text=True, capture_output=True)
        # print(result.stdout)

Predicted class: 0
Predicted class: 0
Predicted class: 0
Predicted class: 1
Predicted class: 1
Predicted class: 1
